In [1]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
import time
import pandas as pd

In [2]:
executable_path={'executable_path':'chromedriver.exe'}

browser = Browser('chrome', **executable_path, headless=False)

# NASA Mars News

In [3]:
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)

In [4]:
html = browser.html
soup = bs(html, 'lxml')

In [5]:
news_title = soup.find('div', class_="content_title").text.strip()

In [6]:
news_title

"NASA's Mars 2020 Rover Closer to Getting Its Name"

In [7]:
news_p = soup.find('div', class_="article_teaser_body").text.strip()

In [8]:
print(news_p)

155 students from across the U.S. have been chosen as semifinalists in NASA's essay contest to name the Mars 2020 rover, and see it launch from Cape Canaveral this July.


In [9]:
'''
table=soup.find('div', class_='grid_layout')
rows=table.find_all('li')
cells=row.find('div', class_="list_text")
news=[]
columns=['date', 'title', 'content']

for row in rows[0:]:
    news_dict={}
    cells=row.find('div', class_="list_text")

    for i, cell in enumerate(cells):
        #print(i, cell.text)
        #print(i, cell.text.strip())
        # The strip() method returns a copy of the string with both leading and trailing characters removed 
        news_dict[columns[i]]=cell.text.strip()
        if i==2:
            news.append(news_dict)

df_news_nasa=pd.DataFrame(news, columns=columns)
#df_news_nasa
'''

'\ntable=soup.find(\'div\', class_=\'grid_layout\')\nrows=table.find_all(\'li\')\ncells=row.find(\'div\', class_="list_text")\nnews=[]\ncolumns=[\'date\', \'title\', \'content\']\n\nfor row in rows[0:]:\n    news_dict={}\n    cells=row.find(\'div\', class_="list_text")\n\n    for i, cell in enumerate(cells):\n        #print(i, cell.text)\n        #print(i, cell.text.strip())\n        # The strip() method returns a copy of the string with both leading and trailing characters removed \n        news_dict[columns[i]]=cell.text.strip()\n        if i==2:\n            news.append(news_dict)\n\ndf_news_nasa=pd.DataFrame(news, columns=columns)\n#df_news_nasa\n'

# JPL Mars Space Images - Featured Image

In [10]:
url='https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)
html = browser.html
soup = bs(html, 'lxml')

In [11]:
image = soup.find("img", class_="thumb")["src"]
featured_image_url = 'https://www.jpl.nasa.gov' + image
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23634-640x350.jpg


# Mars Weather

In [12]:
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)
html = browser.html
soup = bs(html,'lxml')

In [13]:
mars_weather = soup.find('p', class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text').text

print(mars_weather)

InSight sol 406 (2020-01-17) low -100.0ºC (-148.0ºF) high -17.1ºC (1.2ºF)
winds from the SSE at 5.3 m/s (11.8 mph) gusting to 20.7 m/s (46.4 mph)
pressure at 6.40 hPapic.twitter.com/Ws6W7Cf73m


# Mars Facts

In [20]:
# Scrape the table of Mars facts
url = 'https://space-facts.com/mars/'
browser.visit(url)

In [38]:
tables = pd.read_html(url)
df_profile = tables[0]
df_profile

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [39]:
df = df_profile.set_index(0).rename(columns={1: "value"})
del df.index.name
df
#df_profile.set_index('Aspect', inplace=True)

,value
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [15]:
# Convert to HTML table string
df_profile.to_html('mars_profile.html')
dict_profile = df_profile.to_dict()

# Mars Hemispheres

In [3]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)
html = browser.html
soup = bs(html, 'lxml')

In [4]:
mars_hemispheres = []
links = soup.find_all('h3')

for link in links:
    #print(hemi.text)
    mars_hemispheres.append(link.text)

mars_hemispheres

['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

In [6]:
hemisphere_image_urls = []
data = {}

for title in mars_hemispheres:
    
    browser.click_link_by_partial_text(title)
    
    data = {
        'title': title,
        'image' : browser.links.find_by_partial_text('Sample')['href']
    }
    
    print(data)

    hemisphere_image_urls.append(data)
    
    browser.back()


c:\users\alessandra\appdata\local\programs\python\python37-32\lib\site-packages\splinter\driver\webdriver\__init__.py:504: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


{'title': 'Cerberus Hemisphere Enhanced', 'image': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}
{'title': 'Schiaparelli Hemisphere Enhanced', 'image': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}
{'title': 'Syrtis Major Hemisphere Enhanced', 'image': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}
{'title': 'Valles Marineris Hemisphere Enhanced', 'image': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}


In [7]:
print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere Enhanced', 'image': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'image': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'image': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'image': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]
